In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import scipy.optimize
from numpy.random import uniform, normal
import scipy.stats as stats
import sys
from datetime import datetime

## System Functions

In [2]:
def analytical_solution_helper(x, t, parametersVec):
    
    lmd = parametersVec[0];
    ul = parametersVec[1];
    ur = parametersVec[2];
    xr = parametersVec[3];
    
    beta = 2/lmd;
    alpha = (ul - ur)/xr;

    tBreakPoint = 1/(alpha*beta);
    
    if (t <= tBreakPoint):
        if (x < beta*ul*t):
            return ul;
        elif (beta*ul*t <= x <= xr + beta*ur*t):
            return (ul - alpha*x)/(1 - alpha*beta*t);
        else:
            return ur;
    else:
        S = (ul + ur)/lmd;
        xs = S*t + xr/2;
        if (x < xs):
            return ul;
        elif (x == xs):
            return S*lmd/2;
        else:
            return ur;
    

In [3]:
def analytical_solution_ramp(xVec, t, parametersVec):
    
    uVec = np.zeros(xVec.shape);
    for i in range(0, len(xVec)):
        uVec[i] = analytical_solution_helper(xVec[i], t, parametersVec);
        
    return uVec;

In [4]:
def model_prediction_ramp(xVec, t, parametersVec):
    return analytical_solution_ramp(xVec, t, parametersVec);

In [5]:
def shock_position_exact(t, parametersVec):
    lmd = parametersVec[0];
    ul = parametersVec[1];
    ur = parametersVec[2];
    xr = parametersVec[3];
    
    alpha = (ul - ur)/xr;
    beta = 2/lmd;
    
    tBreakPoint = 1/(alpha*beta);
    
    if (t < tBreakPoint):
        return np.round((beta*ul*t + xr + beta*ur*t)/2, 4);
    else:
        S = (ul + ur)/lmd;
        xs = S*t + (ul - ur)/(2*alpha);
        return xs;

## PF helper functions

In [6]:
def exp_obs_shock_pos(t, noiseMean, noiseStd, parametersVec):
    return shock_position_exact(t, parametersVec) + np.random.normal(noiseMean, noiseStd);

In [7]:
def initialParametersVecGenerator(nParticles, nParameters, parametersRange):
    parametersVec = np.zeros((nParticles, nParameters));
    for i in range(0, nParticles):
        for j in range(0, nParameters):
            parametersVec[i, j] = np.random.uniform(parametersRange[j][0], parametersRange[j][1]);

    return parametersVec

## Particle Filter

In [8]:
def ParticleFilter(nParticles, timeStep, prevStateVectors, observation, forwardModelParameters,
                   likelihoodParameters):
#     input: nParticles (number of particles), timeStep, prevStateVectors (matrix comprising of nParticles 
#            previous state vectors as rows, parameters in present case), observation (observations  
#            for the current step), forwardModelParameters, (parameters for the artificial dynamics, covMat),
#            likelihoodParameters (parameters (standard deviation) for the likelihood function)
#     Output: likelihoodVec (likelihood for particles, normalized)
    
    # Propagating the states(artificial dynamics added to each parameter)
    currentStateVectors = prevStateVectors + np.random.multivariate_normal(forwardModelParameters[0], 
                                                                forwardModelParameters[1]/(timeStep*20),
                                                                nParticles);
    # Estimating likelihood 
    likelihoodVec = np.zeros((nParticles, ));
    for i in range(0, nParticles):
        tempParticleObs = shock_position_exact(timeStep, currentStateVectors[i, :]);
        likelihoodVec[i] = stats.norm(observation, likelihoodParameters[0]).pdf(tempParticleObs);
        
    likelihoodVec = likelihoodVec/np.sum(likelihoodVec);
        
    
    return [currentStateVectors, likelihoodVec]

In [9]:
def ResamplingWithArtificialDynamics(nParticles, currentStateVectors, priorVec, likelihoodVec):
#     input: nParticles (number of particles), currentStateVectors (matrix comprising of nParticles 
#            current state vectors as rows, parameters in present case), priorVec (describing   
#            for the current step), likelihoodVec (Likelihood vector from PF)
#     Output: resampledStates (states after resampling), newPosteriorVec (uniform pdf after resampling),
#            frequencyVec (vector indicating how many time a particle is resampled)
    
    posteriorVec = priorVec*likelihoodVec;
    posteriorVec = posteriorVec/np.sum(posteriorVec); # Normalizing the pdf
    
    if(np.round(sum(posteriorVec), 2) != np.round(1, 2)):
        print("error 1 in resampling");

    cumProb = np.cumsum(posteriorVec);
    resampledStates = np.zeros(np.shape(currentStateVectors));
    frequencyVec = np.zeros((nParticles, ));

    uSeed = np.random.uniform()/nParticles;
    k = 0;
    for j in range(0, nParticles):
        tempRandomU = uSeed + (j/nParticles);
        while(tempRandomU > cumProb[k]):
            k = k + 1;
            if(k >= nParticles):
                print("resampling error")
        
        resampledStates[j, :] = currentStateVectors[k, :];
        frequencyVec[k] = frequencyVec[k] + 1;

    newPosteriorVec = np.ones(posteriorVec.size)/nParticles;
    
    return [resampledStates, newPosteriorVec, frequencyVec]


In [17]:
nParticles = 200;
nParameters = 4;

tDomain = [0, 5];
delta_t = 0.05;
tSteps = np.arange(tDomain[0], tDomain[1] + delta_t, delta_t);
tSteps = np.round(tSteps, 4);

parameterOrder = ["lambda", "u_l", "u_r", "x_r"];

lmd_original = 2;
ul_original = 2;
ur_original = 1;
xr_original = 1;

lmdRange = [1.9, 2.4];
ulRange = [2, 2];
urRange = [1, 1];
xrRange = [0.6, 1.1];

parametersOriginal = [lmd_original, ul_original, ur_original, xr_original];
parametersRange = [lmdRange, ulRange, urRange, xrRange];

propagatedParVecMat = np.zeros((len(tSteps), nParticles, nParameters));
resampledParVecMat = np.zeros((len(tSteps), nParticles, nParameters));
particlesFreqMat = np.zeros((len(tSteps), nParticles));
estimatedParMatWeighed = np.zeros((len(tSteps), nParameters));
estimatedParMatResampled = np.zeros((len(tSteps), nParameters));
likelihoodMat = np.zeros((len(tSteps), nParticles));
posteriorMat = np.zeros((len(tSteps), nParticles));
obsShockPos = np.zeros((len(tSteps), ));

if (nParticles == 1000):
    initialParametersVecBasis = initialParametersVecGenerator(nParticles, nParameters, parametersRange);
    
propagatedParVecMat[0,...] = initialParametersVecBasis[0:nParticles];
resampledParVecMat[0,...] = propagatedParVecMat[0,...];
particlesFreqMat[0,...] = np.ones((1, nParticles));
posteriorMat[0,...] = np.ones((1, nParticles))/nParticles;

estimatedParMatWeighed[0, :] = np.matmul(np.transpose(propagatedParVecMat[0,...]), posteriorMat[0,...]);
estimatedParMatResampled[0, :] = estimatedParMatWeighed[0, :];

obsNoiseMean = 0;
obsNoiseStd = 0.2;
likelihoodStd = obsNoiseStd;

artificialDynamicsMean = np.zeros((nParameters, ));
artificialDynamicsCov = np.square(np.diag([(lmdRange[1] - lmdRange[0]),
                                 (ulRange[1] - ulRange[0]),
                                 (urRange[1] - urRange[0]),
                                 (xrRange[1] - xrRange[0])]))/100;

forwardModelParameters = [artificialDynamicsMean, artificialDynamicsCov];
observationParameters = [obsNoiseMean, obsNoiseStd];
likelihoodParameters = [likelihoodStd];

In [18]:
for i in range(1, len(tSteps)):
    timeStep = tSteps[i];
    print(timeStep);
    
    obsShockPos[i] = exp_obs_shock_pos(timeStep, observationParameters[0], observationParameters[1],
                                       parametersOriginal);
    [propagatedParVecMat[i,...], likelihoodMat[i,...]] = ParticleFilter(nParticles, timeStep,
                                                              resampledParVecMat[i - 1,...], obsShockPos[i],
                                                              forwardModelParameters, likelihoodParameters);
    estimatedParMatWeighed[i, :] = np.matmul(np.transpose(propagatedParVecMat[i,...]), likelihoodMat[i,...]);
    
    [resampledParVecMat[i,...], posteriorMat[i,...],
     particlesFreqMat[i,...]] = ResamplingWithArtificialDynamics(nParticles, 
                                                                 propagatedParVecMat[i,...],
                                                                 posteriorMat[i - 1,...], likelihoodMat[i,...]);
    estimatedParMatResampled[i, :] = np.matmul(np.transpose(resampledParVecMat[i,...]), posteriorMat[i,...]);
    

0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
1.0
1.05
1.1
1.15
1.2
1.25
1.3
1.35
1.4
1.45
1.5
1.55
1.6
1.65
1.7
1.75
1.8
1.85
1.9
1.95
2.0
2.05
2.1
2.15
2.2
2.25
2.3
2.35
2.4
2.45
2.5
2.55
2.6
2.65
2.7
2.75
2.8
2.85
2.9
2.95
3.0
3.05
3.1
3.15
3.2
3.25
3.3
3.35
3.4
3.45
3.5
3.55
3.6
3.65
3.7
3.75
3.8
3.85
3.9
3.95
4.0
4.05
4.1
4.15
4.2
4.25
4.3
4.35
4.4
4.45
4.5
4.55
4.6
4.65
4.7
4.75
4.8
4.85
4.9
4.95
5.0


In [19]:
filename1 = 'ex1_' + str(nParticles) + '_' + datetime.today().strftime('%d-%m-%y') + '_artificialDynamics_jackKnife.mat';
filename2 = 'ex1_' + str(nParticles) + '_' + datetime.today().strftime('%d-%m-%y') + '_auxilliary_artificialDynamics_jackKnife.mat';

scipy.io.savemat(filename1, dict(nParameters = nParameters, nParticles = nParticles, tSteps = tSteps, 
                                parametersRange = parametersRange, parametersOriginal = parametersOriginal,
                                resampledParVecMat = resampledParVecMat, posteriorMat = posteriorMat, 
                                estimatedParMatResampled = estimatedParMatResampled, obsNoiseStd = obsNoiseStd,
                                parameterOrder = parameterOrder, obsShockPos = obsShockPos, 
                                likelihoodStd = likelihoodStd, artificialDynamicsCov = artificialDynamicsCov))
                                
                                
                                
scipy.io.savemat(filename2, dict(propagatedParVecMat = propagatedParVecMat, particlesFreqMat = particlesFreqMat,
                                estimatedParMatWeighed = estimatedParMatWeighed, likelihoodMat = likelihoodMat))                        

In [ ]:
plt.plot(tSteps, )